Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [10]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [12]:
num_steps = 2001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 200 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 2.774686
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 200: 0.728619
Minibatch accuracy: 75.0%
Validation accuracy: 78.2%
Minibatch loss at step 400: 0.262143
Minibatch accuracy: 93.8%
Validation accuracy: 79.7%
Minibatch loss at step 600: 0.313379
Minibatch accuracy: 87.5%
Validation accuracy: 81.2%
Minibatch loss at step 800: 0.654639
Minibatch accuracy: 75.0%
Validation accuracy: 82.1%
Minibatch loss at step 1000: 0.316891
Minibatch accuracy: 93.8%
Validation accuracy: 82.5%
Minibatch loss at step 1200: 1.187742
Minibatch accuracy: 56.2%
Validation accuracy: 83.8%
Minibatch loss at step 1400: 0.416653
Minibatch accuracy: 87.5%
Validation accuracy: 83.7%
Minibatch loss at step 1600: 0.873171
Minibatch accuracy: 81.2%
Validation accuracy: 83.5%
Minibatch loss at step 1800: 0.319025
Minibatch accuracy: 81.2%
Validation accuracy: 85.1%
Minibatch lo

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [20]:
graph_cnn_max_pooling = tf.Graph()

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 512

with graph_cnn_max_pooling.as_default():
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
      # Variables.
    layer1_filters = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_filters = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    def cnn_model_with_max_pooling(data):
        conv = tf.nn.conv2d(data, layer1_filters, [1, 1, 1, 1], padding = 'SAME')
        conv_1 = tf.nn.relu(conv + layer1_biases)

        # max pooling. pool stride_size = 2, pooling_size = 2
        pool_1 = tf.nn.max_pool(conv_1, [1, 2, 2, 1], [1, 2, 2, 1], padding = 'SAME')

        conv = tf.nn.conv2d(pool_1, layer2_filters, [1, 1, 1, 1], padding = 'SAME')
        conv_2 = tf.nn.relu(conv + layer2_biases)

        # max pooling again
        pool_2 = tf.nn.max_pool(conv_2, [1, 2, 2, 1], [1, 2, 2, 1], padding = 'SAME')

        shape = pool_2.get_shape().as_list()
        reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])

        fc_nn_hidden_output = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        
        logits = tf.matmul(fc_nn_hidden_output, layer4_weights) + layer4_biases
        
        return logits
    
      # Training computation.
    logits = cnn_model_with_max_pooling(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.AdamOptimizer(1e-3).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(cnn_model_with_max_pooling(tf_valid_dataset))
    test_prediction = tf.nn.softmax(cnn_model_with_max_pooling(tf_test_dataset))

In [21]:
num_steps = 2001

with tf.Session(graph=graph_cnn_max_pooling) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 8.488219
Minibatch accuracy: 6.2%
Validation accuracy: 8.4%
Minibatch loss at step 500: 0.627610
Minibatch accuracy: 93.8%
Validation accuracy: 81.9%
Minibatch loss at step 1000: 0.257921
Minibatch accuracy: 93.8%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 0.759099
Minibatch accuracy: 75.0%
Validation accuracy: 86.4%
Minibatch loss at step 2000: 0.117849
Minibatch accuracy: 93.8%
Validation accuracy: 85.6%
Test accuracy: 92.0%


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [5]:
def get_scope_variable(scope_name, variable_name, shape, initializer):
    with tf.variable_scope(scope_name) as scope:
        try:
            variable = tf.get_variable(variable_name, shape, initializer = initializer)
            return variable
        except ValueError:
            scope.reuse_variables()
            variable = tf.get_variable(variable_name, shape)
            return variable

In [6]:
def cnn(data, patch_size, num_channels, depth):
    filters = get_scope_variable("cnn_layer_1", "filters", [patch_size, patch_size, num_channels, depth], initializer = tf.truncated_normal_initializer(stddev=0.1))
    biases = get_scope_variable("cnn_layer_1", "biases", [depth], initializer = tf.zeros_initializer())
    conv = tf.nn.conv2d(data, filters, [1,1,1,1], padding = "SAME")
    conv_output = tf.nn.relu(conv + biases)
    
    pool = tf.nn.max_pool(conv_output, [1,2,2,1], [1,2,2,1], padding = 'SAME')
    
    filters = get_scope_variable("cnn_layer_2", "filters", [patch_size, patch_size, depth, depth], initializer = tf.truncated_normal_initializer(stddev=0.1))
    biases = get_scope_variable("cnn_layer_2", "biases", [depth], initializer = tf.zeros_initializer())
    conv = tf.nn.conv2d(pool, filters, [1,1,1,1], padding = "SAME")
    conv_output = tf.nn.relu(conv + biases)
    
    pool = tf.nn.max_pool(conv_output, [1,2,2,1], [1,2,2,1], padding = "SAME")
    
    cnn_final_layer_shape = pool.get_shape().as_list()
    cnn_output_1d_tensor = tf.reshape(pool, [cnn_final_layer_shape[0], cnn_final_layer_shape[1] * cnn_final_layer_shape[2] * cnn_final_layer_shape[3]])
    
    return cnn_output_1d_tensor

def fc_nn(cnn_output_1d_tensor, relu_counts, num_labels, drop_out_keep_prob):
    cnn_output_tensor_shape = cnn_output_1d_tensor.get_shape().as_list()
    
    weights = get_scope_variable("fc_nn_hidden_layer", "weights", [cnn_output_tensor_shape[1], relu_counts], initializer = tf.truncated_normal_initializer(stddev=0.1))
    biases = get_scope_variable("fc_nn_hidden_layer", "biases", [relu_counts], initializer = tf.zeros_initializer())
    
    hidden_layer = tf.matmul(cnn_output_1d_tensor, weights) + biases
    hidden_layer_relu = tf.nn.relu(hidden_layer)
    
    hidden_layer_relu_dropout = tf.nn.dropout(hidden_layer_relu, drop_out_keep_prob)
    
    weights = get_scope_variable("fc_nn_output_layer", "weights", [relu_counts, num_labels], initializer = tf.truncated_normal_initializer(stddev=0.1))
    biases = get_scope_variable("fc_nn_output_layer", "biases", [num_labels], initializer = tf.zeros_initializer())
    
    logits = tf.matmul(hidden_layer_relu_dropout, weights) + biases
    
    return logits

def build_model(data, patch_size, num_channels, depth, relu_counts, num_labels, drop_out_keep_prob):
    cnn_output_1d_tensor = cnn(data, patch_size, num_channels, depth)
    logits = fc_nn(cnn_output_1d_tensor, relu_counts, num_labels, drop_out_keep_prob)
    
    return logits

def evaluate_model(data, patch_size, num_channels, depth, relu_counts, num_labels):
    logits = build_model(data, patch_size, num_channels, depth, relu_counts, num_labels, drop_out_keep_prob = 1.0)
    prediction = tf.nn.softmax(logits)
    return prediction

def build_loss_and_optimizer(logits, data_labels, learning_rate):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = data_labels))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    return loss, optimizer

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

num_channels = 1
num_labels = 10
drop_out_keep_prob = 0.75
learning_rate = 0.05

graph_try_for_best = tf.Graph()
with graph_try_for_best.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape = [batch_size, image_size, image_size, num_channels])
    tf_train_labels = tf.placeholder(tf.float32, shape = [batch_size, num_labels])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    logits = build_model(tf_train_dataset, patch_size, num_channels, depth, num_hidden, num_labels, drop_out_keep_prob)
    loss, optimizer = build_loss_and_optimizer(logits, tf_train_labels, learning_rate)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = evaluate_model(tf_valid_dataset, patch_size, num_channels, depth, num_hidden, num_labels)
    test_prediction = evaluate_model(tf_test_dataset, patch_size, num_channels, depth, num_hidden, num_labels)

In [8]:
num_steps = 2001

with tf.Session(graph = graph_try_for_best) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    
    for step in range(num_steps):
        offset = step*batch_size % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
          
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
      
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 2.317371
Minibatch accuracy: 25.0%
Validation accuracy: 16.1%
Minibatch loss at step 500: 0.700920
Minibatch accuracy: 87.5%
Validation accuracy: 81.8%
Minibatch loss at step 1000: 0.296903
Minibatch accuracy: 93.8%
Validation accuracy: 83.6%
Minibatch loss at step 1500: 0.840729
Minibatch accuracy: 81.2%
Validation accuracy: 84.5%
Minibatch loss at step 2000: 0.232523
Minibatch accuracy: 87.5%
Validation accuracy: 85.0%
Test accuracy: 91.4%


Add drop out in fully connected network, Test accuracy: 91.4%